In [647]:
sp1=sphere(10,s=15)
sp2=trns([7,7,7],sphere(5,s=10))

sp3=trns([7,7,7],q_rot(['y45','z45'],sphere(5,s=10)))
sp4=cpo(sp3)
ip1=ip_surf(sp1,sp3)

with open('/Users/sanjeevprabhakar/iCloud Drive (Archive)/Documents/Download/openscad/explanation of process for creating fillet.scad','w+') as f:
    f.write(f'''
//explanation of the process of creating fillet:

//refer bigger sphere as sp1
//refer smaller sphere as sp2

//"Blue" points are the intersection points between sp1 and sp2
//"magenta" line at the normal drawn from the "Blue" points to sp1 surface
//"green" line arc is drawn with axis of rotation "navy blue" line to find the intersection point "green" with sp1. radius of the arc is the radius of the fillet
//"cyan" line arc drawn with axis of rotation "navy blue" line to find the intersection point "cyan" with sp2. radius of the arc is the radius of the fillet
//with 3 points "cyan", "blue" and "green" arcs are drawn to form fillets
    
include<dependencies2.scad> 
for(p={sp1})p_line3dc(p,.05);
for(p={cpo(sp3)})p_line3d(p,.05);
color("blue")points({ip1},.3);
color("magenta")for(p={normals.tolist()})p_line3d(p,.05);
color("green")for(p={cir})p_line3d(p,.02);
color("green")points({pnt3},.2);
color("cyan")for(p={cir1})p_line3d(p,.02);
color("cyan")points({pnt2},.2);
color("navy")for(p={rotation_axis.tolist()})p_line3d(p,.05);
color("black")for(p={sol})p_line3dc(p,.02);

%{swp(sp1)}
%{swp(sp3)}
%{swp(sol1)}
''') 

In [641]:
p,p1,r,s,o=sp1,sp3,2,10,0

pa=[[[[p[i][j],p[i][j+1],p[i+1][j]],[p[i+1][j+1],p[i+1][j],p[i][j+1]]] if j<len(p[0])-1 else \
     [[p[i][j],p[i][0],p[i+1][j]],[p[i+1][0],p[i+1][j],p[i][0]]] \
     for j in range(len(p[0]))] for i in range(len(p)-1)]
pa=array(pa).reshape(-1,3,3)

#     pb=[[[p1[i][j],p1[i+1][j]] for j in range(len(p1[0]))] for i in range(len(p1)-1)]
#     pb=array(pb).reshape(-1,2,3)
p2=cpo(p1)
pb=[[[p2[i][j],p2[i][j+1]] for j in range(len(p2[0])-1)] for i in range(len(p2))]
pb=array(pb).reshape(-1,2,3)

p01,p02,p03,p04,p05=pa[:,0],pa[:,1],pa[:,2],pb[:,0],pb[:,1]

v1,v2,v3=p05-p04,p02-p01,p03-p01
i,j=len(v1),len(v2)
#     array([(-v1).tolist()]*j).transpose(1,0,2).shape,array([v2]*i).shape,array([cross(v2,v3)]*i).shape,(p04[:,None]-p01).shape
#     cross(v3,-v1[:,None]).shape,cross(-v1[:,None],v2).shape
a=einsum('ijk,ijk->ij',array([cross(v2,v3)]*i),p04[:,None]-p01)
b=(1/einsum('ijk,ijk->ij',array([-v1]*j).transpose(1,0,2),array([cross(v2,v3)+[.00001,.00001,.00001]]*i)))
t1=einsum('ij,ij->ij',a,b)

a=einsum('ijk,ijk->ij',cross(v3,-v1[:,None]),p04[:,None]-p01)
b=(1/einsum('ijk,ijk->ij',array([-v1]*j).transpose(1,0,2),array([cross(v2,v3)+[.00001,.00001,.00001]]*i)))
t2=einsum('ij,ij->ij',a,b)

a=einsum('ijk,ijk->ij',cross(-v1[:,None],v2),p04[:,None]-p01)
b=(1/einsum('ijk,ijk->ij',array([-v1]*j).transpose(1,0,2),array([cross(v2,v3)+[.00001,.00001,.00001]]*i)))
t3=einsum('ij,ij->ij',a,b)

condition=(t1>=0) & (t1<=1) & (t2>=0) & (t2<=1) & (t3>=0) & (t3<=1) & (t2+t3>=0) & (t2+t3<=1)
#     condition.shape

pnt1=(p04[:,None]+einsum('ijk,ij->ijk',array([v1]*j).transpose(1,0,2),t1))[condition]
#     pnt1.shape

uv1=v1/linalg.norm(v1,axis=1).reshape(-1,1)
uv1=array([uv1]*j).transpose(1,0,2)[condition]
#     uv1.shape
#     pnt2=pnt1+uv1*r

a=cross(v2,v3)
b=a/(linalg.norm(a,axis=1).reshape(-1,1)+.00001)
b=array([b]*i)[condition]
#     b.shape
normals=array([pnt1,pnt1+b*r]).transpose(1,0,2)
nxt_pnt=array(pnt1[1:].tolist()+[pnt1[0]])
v_rot=nxt_pnt-pnt1
rotation_axis=array([pnt1,pnt1+v_rot*.5]).transpose(1,0,2)
if o==0:
    cir=array([[pnt1[i]+array(q(v_rot[i],b[i]*r,t)) for t in linspace(0,180,5)] for i in arange(len(pnt1))]).tolist()
else:
    cir=array([[pnt1[i]+array(q(v_rot[i],b[i]*r,-t)) for t in linspace(0,180,5)] for i in arange(len(pnt1))]).tolist()

pc=array([[[cir[i][j],cir[i][j+1]]  for j in arange(len(cir[0])-1)] for i in arange(len(cir))]).reshape(-1,2,3)
#     pc.shape

p01,p02,p03,p04,p05=pa[:,0],pa[:,1],pa[:,2],pc[:,0],pc[:,1]

v1,v2,v3=p05-p04,p02-p01,p03-p01
i,j=len(v1),len(v2)

a1=einsum('ijk,ijk->ij',array([cross(v2,v3)]*i),p04[:,None]-p01)
b1=(1/einsum('ijk,ijk->ij',array([-v1]*j).transpose(1,0,2),array([cross(v2,v3)+[.00001,.00001,.00001]]*i)))
t1=einsum('ij,ij->ij',a1,b1)

a1=einsum('ijk,ijk->ij',cross(v3,-v1[:,None]),p04[:,None]-p01)
b1=(1/einsum('ijk,ijk->ij',array([-v1]*j).transpose(1,0,2),array([cross(v2,v3)+[.00001,.00001,.00001]]*i)))
t2=einsum('ij,ij->ij',a1,b1)

a1=einsum('ijk,ijk->ij',cross(-v1[:,None],v2),p04[:,None]-p01)
b1=(1/einsum('ijk,ijk->ij',array([-v1]*j).transpose(1,0,2),array([cross(v2,v3)+[.00001,.00001,.00001]]*i)))
t3=einsum('ij,ij->ij',a1,b1)

condition=(t1>=0) & (t1<=1) & (t2>=0) & (t2<=1) & (t3>=0) & (t3<=1) & (t2+t3>=0) & (t2+t3<=1)
#     condition.shape

pnt3=(p04[:,None]+einsum('ijk,ij->ijk',array([v1]*j).transpose(1,0,2),t1))[condition]
pnt3=sort_pointsv(pnt1,pnt3)

#     if o==0:
cir1=array([[pnt1[i]+array(q(v_rot[i],b[i]*r,-t)) for t in linspace(-90,90,5)] for i in arange(len(pnt1))]).tolist()
#     else:
#         cir=array([[pnt1[i]+array(q(v_rot[i],b[i]*r,t)) for t in linspace(-90,90,5)] for i in arange(len(pnt1))]).tolist()

pa=[[[[p1[i][j],p1[i][j+1],p1[i+1][j]],[p1[i+1][j+1],p1[i+1][j],p1[i][j+1]]] if j<len(p1[0])-1 else \
     [[p1[i][j],p1[i][0],p1[i+1][j]],[p1[i+1][0],p1[i+1][j],p1[i][0]]] \
     for j in range(len(p1[0]))] for i in range(len(p1)-1)]
pa=array(pa).reshape(-1,3,3)

pc=array([[[cir1[i][j],cir1[i][j+1]]  for j in arange(len(cir1[0])-1)] for i in arange(len(cir1))]).reshape(-1,2,3)

p01,p02,p03,p04,p05=pa[:,0],pa[:,1],pa[:,2],pc[:,0],pc[:,1]

v1,v2,v3=p05-p04,p02-p01,p03-p01
i,j=len(v1),len(v2)

a1=einsum('ijk,ijk->ij',array([cross(v2,v3)]*i),p04[:,None]-p01)
b1=(1/einsum('ijk,ijk->ij',array([-v1]*j).transpose(1,0,2),array([cross(v2,v3)+[.00001,.00001,.00001]]*i)))
t1=einsum('ij,ij->ij',a1,b1)

a1=einsum('ijk,ijk->ij',cross(v3,-v1[:,None]),p04[:,None]-p01)
b1=(1/einsum('ijk,ijk->ij',array([-v1]*j).transpose(1,0,2),array([cross(v2,v3)+[.00001,.00001,.00001]]*i)))
t2=einsum('ij,ij->ij',a1,b1)

a1=einsum('ijk,ijk->ij',cross(-v1[:,None],v2),p04[:,None]-p01)
b1=(1/einsum('ijk,ijk->ij',array([-v1]*j).transpose(1,0,2),array([cross(v2,v3)+[.00001,.00001,.00001]]*i)))
t3=einsum('ij,ij->ij',a1,b1)

condition=(t1>=0) & (t1<=1) & (t2>=0) & (t2<=1) & (t3>=0) & (t3<=1) & (t2+t3>=0) & (t2+t3<=1)
#     condition.shape

pnt2=(p04[:,None]+einsum('ijk,ij->ijk',array([v1]*j).transpose(1,0,2),t1))[condition]
pnt2=sort_pointsv(pnt1,pnt2)


sol=array([pnt3,pnt1,pnt2]).transpose(1,0,2)
sol=[fillet_3p_3d(p3,p2,p1,r_3p_3d([p1,p2,p3])*1.9,s) for (p1,p2,p3) in sol]
sol1=sol+[sol[0]]
